In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import random
from tqdm import tqdm
from copy import deepcopy

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import TapasForSequenceClassification, TapasTokenizer

import numpy as np 
import pandas as pd


In [95]:
# https://www.kaggle.com/datasets/rohitsahoo/employee
data = pd.read_csv('/Users/ywkim/Downloads/archive/train.csv', )
test = pd.read_csv('/Users/ywkim/Downloads/archive/test.csv', )

In [93]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = .2, random_state = 42, stratify = data['Attrition'])
val, test = train_test_split(test, test_size = .5, random_state = 42, stratify = test['Attrition'])

In [58]:
str_cols = train.select_dtypes(object).columns.tolist()

In [53]:
cat_to_str = {
    'Education':dict(zip(range(1,6), ['Below College',
'College',
'Bachelor',
'Master',
'Doctor'])),


'EnvironmentSatisfaction':dict(zip(range(1,5), [
    'Low',
'Medium',
'High',
'Very High'
])),


'JobInvolvement':dict(zip(range(1,5), [
    'Low',
'Medium',
'High',
'Very High'
])),

'JobSatisfaction':dict(zip(range(1,5), [
'Low',
'Medium',
'High',
'Very High'
])),

'PerformanceRating':dict(zip(range(1,5), [
'Low',
'Good',
'Excellent',
'Outstanding'
])),

'RelationshipSatisfaction':dict(zip(range(1,5), [
'Low',
'Medium',
'High',
'Very High'
])),


'WorkLifeBalance':dict(zip(range(1,5), [
'Bad',
'Good',
'Better',
'Best'
]))

}

In [62]:
for col in cat_to_str.keys():
    training[col] = training[col].apply(lambda x: cat_to_str[col][x]).copy()
    val[col] = val[col].apply(lambda x: cat_to_str[col][x]).copy()

In [50]:
import re
re.sub(r'[\(\)\{\}\']', '', str(dict(train.loc[0])))

'Age: 41, Attrition: 1, BusinessTravel: Travel_Rarely, DailyRate: 1102, Department: Sales, DistanceFromHome: 1, Education: 2, EducationField: Life Sciences, EmployeeCount: 1, EmployeeNumber: 1, EnvironmentSatisfaction: 2, Gender: Female, HourlyRate: 94, JobInvolvement: 3, JobLevel: 2, JobRole: Sales Executive, JobSatisfaction: 4, MaritalStatus: Single, MonthlyIncome: 5993, MonthlyRate: 19479, NumCompaniesWorked: 8, Over18: Y, OverTime: Yes, PercentSalaryHike: 11, PerformanceRating: 3, RelationshipSatisfaction: 1, StandardHours: 80, StockOptionLevel: 0, TotalWorkingYears: 8, TrainingTimesLastYear: 0, WorkLifeBalance: 1, YearsAtCompany: 6, YearsInCurrentRole: 4, YearsSinceLastPromotion: 0, YearsWithCurrManager: 5'

In [65]:
training_text = []
X_training = training.drop('Attrition', axis=1).copy()

for idx, row in X_training.iterrows():
    training_text.append(re.sub(r'[\(\)\{\}\']', '', str(dict(row))))


In [74]:
training_text[0:1]

['Age: 34, BusinessTravel: Travel_Rarely, DailyRate: 507, Department: Sales, DistanceFromHome: 15, Education: College, EducationField: Medical, EmployeeCount: 1, EmployeeNumber: 1356, EnvironmentSatisfaction: High, Gender: Female, HourlyRate: 66, JobInvolvement: High, JobLevel: 2, JobRole: Sales Executive, JobSatisfaction: Low, MaritalStatus: Single, MonthlyIncome: 6125, MonthlyRate: 23553, NumCompaniesWorked: 1, Over18: Y, OverTime: No, PercentSalaryHike: 12, PerformanceRating: Excellent, RelationshipSatisfaction: Very High, StandardHours: 80, StockOptionLevel: 0, TotalWorkingYears: 10, TrainingTimesLastYear: 6, WorkLifeBalance: Best, YearsAtCompany: 10, YearsInCurrentRole: 8, YearsSinceLastPromotion: 9, YearsWithCurrManager: 6']

In [ ]:
tokenizer(train.astype(str), training_text[0:1], max_length=512, truncation=True, )

In [51]:
train.columns[train.nunique()==1]

Index(['EmployeeCount', 'Over18', 'StandardHours'], dtype='object')

In [3]:
model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [5]:
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-tabfact")
# model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")